In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import requests
import json
import tweepy
import io
import os
from dotenv import find_dotenv, load_dotenv
import time

In [2]:
df_tae = pd.read_csv('twitter-archive-enhanced.csv')

df_tae.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
583,800188575492947969,NaN,NaN,2016-11-20 04:06:37 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Bo. He's a Benedoop Cum...,6.816941e+17,4.196984e+09,2015-12-29 04:31:49 +0000,https://twitter.com/dog_rates/status/681694085...,11,10,Bo,NaN,NaN,pupper,NaN
1850,675740360753160193,NaN,NaN,2015-12-12 18:13:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a pupper licking in slow motion. 12/10 ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675740360...,12,10,NaN,NaN,NaN,pupper,NaN
886,759943073749200896,NaN,NaN,2016-08-01 02:45:22 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",Here's a wicked fast pupper. 12/10 camera coul...,NaN,NaN,NaN,https://vine.co/v/5AJm5pq7Kav,12,10,NaN,NaN,NaN,pupper,NaN
1353,703769065844768768,NaN,NaN,2016-02-28 02:29:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you're trying to watch your favorite tv s...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/703769065...,10,10,NaN,NaN,NaN,NaN,NaN
2287,667177989038297088,NaN,NaN,2015-11-19 03:10:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Dasani Kingfisher from Maine. His na...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667177989...,8,10,a,NaN,NaN,NaN,NaN


In [3]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

page = requests.get(url)
df_img = pd.read_csv(io.StringIO(page.content.decode('utf-8')), sep='\t')

with open('image-predictions.tsv', 'wb') as file:
    file.write(page.content)

df_img.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1962,866686824827068416,https://pbs.twimg.com/media/DAcXEWuXkAIBDGJ.jpg,1,flat-coated_retriever,0.514730,True,groenendael,0.306407,True,curly-coated_retriever,0.061314,True
1559,793256262322548741,https://pbs.twimg.com/media/CwI2XCvXEAEO8mc.jpg,1,basset,0.207622,True,Walker_hound,0.060574,True,beagle,0.041221,True
91,667544320556335104,https://pbs.twimg.com/media/CUOYBbbWIAAXQGU.jpg,1,Pomeranian,0.412893,True,Pembroke,0.312958,True,Chihuahua,0.071960,True
839,694356675654983680,https://pbs.twimg.com/media/CaLZtmsWQAApbFw.jpg,1,hamster,0.429871,False,Pomeranian,0.144272,True,pretzel,0.127220,False
1146,730573383004487680,https://pbs.twimg.com/media/CiOEnI6WgAAmq4E.jpg,2,American_Staffordshire_terrier,0.810158,True,Labrador_retriever,0.058205,True,Weimaraner,0.027929,True


In [4]:
dotenv_path = find_dotenv()
if dotenv_path:
    load_dotenv(dotenv_path)
    
# Load credentials from environment variables
bearer_token_ = os.getenv('BEARER_TOKEN')
consumer_key_ = os.getenv('CONSUMER_KEY')
consumer_secret_ = os.getenv('CONSUMER_SECRET')
access_token_ = os.getenv('ACCESS_TOKEN')
access_secret_ = os.getenv('ACCESS_SECRET')

client = tweepy.Client(bearer_token=bearer_token_,
    consumer_key=consumer_key_,
    consumer_secret=consumer_secret_,
    access_token=access_token_,
    access_token_secret=access_secret_,
    wait_on_rate_limit=True
    )

In [ ]:
tweet_ids = df_img['tweet_id'].tolist()

# Process in batches of 100 so it works for the get_tweets method
for i in range(0, len(tweet_ids), 100):
    try:
        tweets = client.get_tweets(tweet_ids[i:i+100], tweet_fields=['public_metrics','text'])
        if tweets.data:
            for tweet in tweets.data:
                file = open('tweet_json.txt', 'a')
                file.write(json.dumps(tweet.data) + '\n')
                file.close()
        else:
            print(f"No data returned for batch {i}")
            
    except tweepy.TooManyRequests as e:
        print("Rate limit reached. Retrying after 15 mins and 5 secs...")
        time.sleep(15 * 61)  # Wait 15 minutes and 5 secs.

    except Exception as e:
        print("An error occurred: " + str(e))

An error occurred: maximum recursion depth exceeded
An error occurred: maximum recursion depth exceeded
